<a href="https://www.kaggle.com/code/lukalafaye/hub-eau-dataset?scriptVersionId=211349664" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [4]:
import requests
import pandas as pd
from io import StringIO

def fetch_all_data(url, output_file, params, date_key_api_min, date_key_api_max, date_key, desc=False, json=False):
    size = params.get('size', 10)
    done = True

    # Make the initial request
    response = requests.get(url, params)
    print("Response obtained.")
    if response.status_code not in [200, 206]:
        done = False
        print(f"Error: {response.status_code} - {response.text}")
        return 

    if json:
        data = response.json()
        df = pd.json_normalize(data['data'])
    else:
        content = response.content.decode('utf-8')
        df = pd.read_csv(StringIO(content), sep=";")

    df.to_csv(output_file, index=False, sep=";")

    print("before while")
    while True:
        stop = len(df)
        if stop == 0 or stop < size:
            done = True
            break

        last_date = df[date_key].iloc[-1]
        
        last_date = pd.to_datetime(last_date)
        print(last_date)


        if not desc:
            next_min_date = last_date + pd.Timedelta(days=1)
            params[date_key_api_min] = next_min_date.strftime('%Y-%m-%d')
        else:
            next_max_date = last_date - pd.Timedelta(days=1)
            params[date_key_api_max] = next_max_date.strftime('%Y-%m-%d')

        response = requests.get(url, params)
        if response.status_code not in [200, 206]:
            done = False
            print(f"Error: {response.status_code} - {response.text}")
            break

        if json:
            data = response.json()
            df = pd.json_normalize(data['data'])
        else:
            content = response.content.decode('utf-8')
            df = pd.read_csv(StringIO(content), sep=";")

        if len(df) == 0:
            done = True
            break

        df.to_csv(output_file, mode='a', index=False, header=False, sep=";")

    print("DONE", done)
    return


In [5]:
params = {
    "size": 20000,
    "code_departement": "75,77,78,91,92,93,94,95",
    "fields": (
        "code_station,libelle_station,code_commune,libelle_commune,latitude,"
        "longitude,code_ecoulement,libelle_ecoulement,date_observation"
    ),
    "date_observation_min": "2014-01-01T00:00:00",
    "date_observation_max": "2024-12-31T23:59:59"
}

date_key_api_min = "date_observation_min"
date_key_api_max = "date_observation_max"
date_key = "date_observation"

url = "https://hubeau.eaufrance.fr/api/v1/ecoulement/observations"
output_file = "analyses_ec.csv"

fetch_all_data(
    url,
    output_file,
    params,
    date_key_api_min,
    date_key_api_max,
    date_key,
    desc=False,
    json=True
)

Response obtained.
before while
DONE True


In [ ]:
params = {
    "size": 20000,
    "fields": (
        "code_commune,nom_commune,date_prelevement,libelle_parametre,"
        "resultat_numerique,libelle_unite,conclusion_conformite_prelevement"
    ),
    "code_departement": "75,77,78,91,92,93,94,95",
    "date_min_prelevement": "2014-01-01T00:00:00",
    "date_max_prelevement": "2024-12-31T23:59:59",
}

date_key_api_min = "date_min_prelevement"
date_key_api_max = "date_max_prelevement"
date_key = "date_prelevement"

url = "https://hubeau.eaufrance.fr/api/v1/qualite_eau_potable/resultats_dis"

output_file = "analyses_qu.csv"

fetch_all_data(
    url,
    output_file,
    params,
    date_key_api_min,
    date_key_api_max,
    date_key,
    desc=True,
    json=True
)

In [ ]:
params = {
    "size": 20000,
    "fields": (
        "code_lieusurv,libelle_lieusurv,longitude,latitude,"
        "date_prel,code_parametre,libelle_parametre,"
        "resultat_analyse,libelle_unite_resultat"
    ),
    "date_min_prel": "2014-01-01T00:00:00",
    "date_max_prel": "2024-12-31T23:59:59",
}

date_key_api_min = "date_min_prel"
date_key_api_max = "date_max_prel"
date_key = "date_prel"

url = "https://hubeau.eaufrance.fr/api/v1/surveillance_littoral/contaminants_chimiques.csv"

output_file = "analyses_co.csv"

fetch_all_data(
    url,
    output_file,
    params,
    date_key_api_min,
    date_key_api_max,
    date_key,
    desc=True,
    json=False
)

In [ ]:
df_qualite = pd.read_csv("analyses_qu.csv", sep=";")
df_qualite = pd.DataFrame(all_data)
df_qualite = df_qualite[df_qualite["libelle_unite"] != "SANS OBJET"]
df_qualite["conclusion_conformite_prelevement"] = df_qualite["conclusion_conformite_prelevement"].apply(
    lambda x: 1 if "Eau d'alimentation conforme" in x else 0
)
df_qualite.head()

In [ ]:
df_ecoulement = pd.read_csv("analyses_ec.csv", sep=";")
df_littoral = pd.read_csv("analyses_co.csv", sep=";")

In [ ]:
df_ecoulement.head()

In [ ]:
df_littoral.head()